In [25]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
import requests
from bs4 import BeautifulSoup

In [26]:
llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    temperature=0.6,
    timeout=None,
    max_retries=3,
    api_key="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",
)

In [27]:
def data_from(drug):
    
    url = f"https://www.drugs.com/{llm.invoke(f"{drug}'s generic name only").content.lower()}.html"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        content = soup.find('div', id='content')
    
        text = content.get_text(separator=" ", strip=True)
        return text

In [33]:
refine_prompt = PromptTemplate.from_template(
    template='''### SCRAPED TEXT: {webdata}
    ### TASK INSTRUCTIONS:
    The above given data is the text data scrapped from a website. It contains the drugs (medicine) information.
    Format the given data in a structured way and summarise the repetatives without loosing leaving any important information.'''
)

In [34]:
extract_prompt = PromptTemplate.from_template(
    template='''
### TEXT: {page}

### TASK INSTRUCTIONS:
Based on the above structured content, please extract and return information about the drug in the following categories. For each, list specific details provided, or use "not specified" if no information is found and no other details addition. 

### REQUIRED INFORMATION:
1. **Drug Name** - Primary name of the drug and common brand names.
2. **Recommended Foods** - List any foods recommended when taking this drug (if none, specify "none").
3. **Foods to Avoid** - List any foods that should be avoided when taking this drug.
4. **Interacting Drugs** - Provide a list of other drugs that interact with this drug. If a specific waiting period is mentioned for timing between medications, include the wait time in hours; if none is specified, use -1.
5. **Common Side Effects** - Symptoms commonly reported by patients using this drug.
6. **Serious Side Effects** - Symptoms or reactions that may require immediate medical attention.
7. **Primary Uses** - The main medical conditions this drug is prescribed to treat.

### OUTPUT FORMAT:
Return the information in STRICT JSON format, following this example:

{{
    "drug": "ExampleDrugName",
    "super_food": ["list any good foods here"],
    "bad_food": ["list any foods to avoid here"],
    "interactive_drug": [
        {{ "name": "OtherDrugName", "delta": integer in hours or -1 }}
    ],
    "common_symptom": ["list common symptoms here"],
    "serious_symptom": ["list serious symptoms here"],
    "treatment": ["main treatments or conditions for this drug"]
}}
### IMPORTANT:
- Output **only JSON** with no extra commentary or explanation.
- If a category has no information, use "not specified" or an empty list as applicable.

''')


In [35]:
chain_refine = refine_prompt | llm
chain_extract = extract_prompt | llm
refine_data = chain_refine.invoke(input=data_from("thyroxine")).content
data = chain_extract.invoke(input=refine_data).content

In [36]:
print(refine_data)

### Levothyroxine Medication Information

#### Overview

* **Generic Name:** Levothyroxine
* **Brand Names:** Ermeza, Euthyrox, Levo-T, Levoxyl, Synthroid, Thyquidity, Tirosint, Tirosint-Sol, Unithroid
* **Drug Class:** Thyroid drugs

#### Uses

* Treats hypothyroidism (underactive thyroid) in adults and children
* Used in adults along with surgery and radioactive iodine therapy to treat a certain type of thyroid cancer
* Used in adults to treat myxedema coma

#### Side Effects

* **Serious Side Effects:**
	+ Sudden pain or trouble moving your hip, wrist, or back
	+ Fast or irregular heartbeats
	+ Chest pain, pain spreading to your jaw or shoulder
	+ Wheezing
	+ Fever, swollen glands, itching, joint pain, or not feeling well
	+ Nausea, vomiting, or diarrhea
	+ High blood sugar - increased thirst, increased urination, dry mouth, fruity breath odor
* **Common Side Effects:**
	+ Fever, hot flashes, increased sweating
	+ Tiredness
	+ Skin rash, hair loss
	+ Chest pain, fast or irregular he

In [37]:
print(data)

{
    "drug": "Levothyroxine",
    "super_food": [],
    "bad_food": ["milk"],
    "interactive_drug": [
        { "name": "Calcium carbonate", "delta": 4 },
        { "name": "Sevelamer, lanthanum", "delta": 4 },
        { "name": "Cholestyramine, colesevelam, colestipol", "delta": 4 },
        { "name": "Iron supplements", "delta": 4 },
        { "name": "Sucralfate", "delta": 4 },
        { "name": "Sodium polystyrene sulfonate (Kalexate, Kayexalate, Kionex)", "delta": 4 },
        { "name": "Stomach acid reducers - esomeprazole, lansoprazole, omeprazole, rabeprazole, Nexium, Prilosec, Prevacid, Protonix, Zegerid, and others", "delta": 4 },
        { "name": "Antacids that contain aluminum or magnesium - Gaviscon, Maalox, Milk of Magnesia, Mintox, Mylanta, Pepcid Complete, and others", "delta": 4 },
        { "name": "Orlistat", "delta": -1 },
        { "name": "Phenobarbital, rifampin", "delta": -1 },
        { "name": "Ketamine, steroid medicines", "delta": -1 },
        { "name":